# Kalman Filters: From Fundamentals to Advanced Filtering

This notebook provides a comprehensive introduction to Kalman filtering using the Tracker Component Library (pytcl). We cover:

1. **Linear Kalman Filter (KF)** - Optimal estimation for linear systems
2. **Extended Kalman Filter (EKF)** - First-order approximation for nonlinear systems
3. **Unscented Kalman Filter (UKF)** - Sigma-point methods for better nonlinear handling
4. **Square-Root Filters** - Numerically stable variants
5. **Interacting Multiple Model (IMM)** - Multi-model estimation

## Prerequisites

```bash
pip install nrl-tracker matplotlib numpy
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pytcl.dynamic_estimation import (
    kf_predict, kf_update,
    ekf_predict, ekf_update,
    ukf_predict, ukf_update,
    sr_kf_predict, sr_kf_update,
)
from pytcl.dynamic_models import (
    f_constant_velocity_2d, q_constant_velocity_2d,
    f_coordinated_turn_2d, q_coordinated_turn_2d,
)

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

ImportError: cannot import name 'sr_kf_predict' from 'pytcl.dynamic_estimation' (/Users/nedonatelli/Documents/Local Repositories/TCL/pytcl/dynamic_estimation/__init__.py)

## 1. Linear Kalman Filter

The Kalman filter is the optimal estimator for linear Gaussian systems. It consists of two steps:

**Prediction:**
$$\hat{x}_{k|k-1} = F \hat{x}_{k-1|k-1}$$
$$P_{k|k-1} = F P_{k-1|k-1} F^T + Q$$

**Update:**
$$K = P_{k|k-1} H^T (H P_{k|k-1} H^T + R)^{-1}$$
$$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K(z - H\hat{x}_{k|k-1})$$
$$P_{k|k} = (I - KH) P_{k|k-1}$$

### Example: Tracking a Moving Object

In [ ]:
# System parameters
dt = 0.1  # Time step
n_steps = 100

# State: [x, vx, y, vy]
# Constant velocity model
F = np.array([
    [1, dt, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, dt],
    [0, 0, 0, 1]
])

# Process noise
q = 0.1  # Process noise intensity
Q = q * np.array([
    [dt**3/3, dt**2/2, 0, 0],
    [dt**2/2, dt, 0, 0],
    [0, 0, dt**3/3, dt**2/2],
    [0, 0, dt**2/2, dt]
])

# Measurement model: observe position only
H = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0]
])

# Measurement noise
R = np.eye(2) * 1.0

print(f"State dimension: {F.shape[0]}")
print(f"Measurement dimension: {H.shape[0]}")

In [ ]:
# Generate ground truth trajectory
true_state = np.array([0.0, 1.0, 0.0, 0.5])  # Start at origin, moving diagonally
true_states = [true_state.copy()]
measurements = []

for _ in range(n_steps):
    # Propagate true state
    true_state = F @ true_state + np.random.multivariate_normal(np.zeros(4), Q)
    true_states.append(true_state.copy())
    
    # Generate measurement
    z = H @ true_state + np.random.multivariate_normal(np.zeros(2), R)
    measurements.append(z)

true_states = np.array(true_states)
measurements = np.array(measurements)

print(f"Generated {len(measurements)} measurements")

In [ ]:
# Run Kalman filter
x_est = np.array([0.0, 0.0, 0.0, 0.0])  # Initial estimate
P_est = np.eye(4) * 10.0  # Initial covariance (uncertain)

estimates = [x_est.copy()]
covariances = [P_est.copy()]

for z in measurements:
    # Predict
    pred = kf_predict(x_est, P_est, F, Q)
    x_pred, P_pred = pred.x, pred.P
    
    # Update
    upd = kf_update(x_pred, P_pred, z, H, R)
    x_est, P_est = upd.x, upd.P
    
    estimates.append(x_est.copy())
    covariances.append(P_est.copy())

estimates = np.array(estimates)
covariances = np.array(covariances)

print(f"Final position estimate: ({estimates[-1, 0]:.2f}, {estimates[-1, 2]:.2f})")
print(f"Final position uncertainty (1σ): ({np.sqrt(covariances[-1, 0, 0]):.3f}, {np.sqrt(covariances[-1, 2, 2]):.3f})")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trajectory plot
ax = axes[0]
ax.plot(true_states[:, 0], true_states[:, 2], 'g-', linewidth=2, label='True trajectory')
ax.scatter(measurements[:, 0], measurements[:, 1], c='r', s=10, alpha=0.5, label='Measurements')
ax.plot(estimates[:, 0], estimates[:, 2], 'b--', linewidth=2, label='KF estimate')
ax.set_xlabel('X position')
ax.set_ylabel('Y position')
ax.set_title('Kalman Filter Tracking')
ax.legend()
ax.axis('equal')

# Position error over time
ax = axes[1]
pos_error = np.sqrt((estimates[1:, 0] - true_states[1:, 0])**2 + 
                     (estimates[1:, 2] - true_states[1:, 2])**2)
pos_uncertainty = np.sqrt(covariances[1:, 0, 0] + covariances[1:, 2, 2])

time = np.arange(len(pos_error)) * dt
ax.plot(time, pos_error, 'b-', label='Position error')
ax.fill_between(time, 0, 2*pos_uncertainty, alpha=0.3, label='2σ uncertainty')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Error (m)')
ax.set_title('Estimation Error vs Uncertainty')
ax.legend()

plt.tight_layout()
plt.show()

## 2. Extended Kalman Filter (EKF)

For nonlinear systems, the EKF linearizes about the current estimate:

$$x_{k+1} = f(x_k) + w_k$$
$$z_k = h(x_k) + v_k$$

The Jacobians $F = \frac{\partial f}{\partial x}$ and $H = \frac{\partial h}{\partial x}$ are used in place of the linear matrices.

### Example: Tracking with Range-Bearing Measurements

In [ ]:
# Nonlinear measurement model: range and bearing from origin
def h_radar(x):
    """Radar measurement: range and bearing."""
    px, vx, py, vy = x
    r = np.sqrt(px**2 + py**2)
    theta = np.arctan2(py, px)
    return np.array([r, theta])

def H_radar(x):
    """Jacobian of radar measurement."""
    px, vx, py, vy = x
    r = np.sqrt(px**2 + py**2)
    if r < 1e-10:
        r = 1e-10
    return np.array([
        [px/r, 0, py/r, 0],
        [-py/r**2, 0, px/r**2, 0]
    ])

# Measurement noise for range-bearing
R_radar = np.diag([0.5**2, np.radians(2)**2])  # 0.5m range, 2 deg bearing

# Generate radar measurements
radar_measurements = []
for state in true_states[1:]:
    z_true = h_radar(state)
    z = z_true + np.random.multivariate_normal(np.zeros(2), R_radar)
    radar_measurements.append(z)

radar_measurements = np.array(radar_measurements)

In [ ]:
# Run EKF with radar measurements
x_ekf = np.array([1.0, 0.0, 1.0, 0.0])  # Start with offset estimate
P_ekf = np.eye(4) * 10.0

# Linear dynamics (constant velocity)
def f_cv(x):
    return F @ x

ekf_estimates = [x_ekf.copy()]

for z in radar_measurements:
    # Predict (linear dynamics)
    pred = ekf_predict(x_ekf, P_ekf, f_cv, F, Q)
    x_pred, P_pred = pred.x, pred.P
    
    # Update (nonlinear measurement)
    upd = ekf_update(x_pred, P_pred, z, h_radar, H_radar(x_pred), R_radar)
    x_ekf, P_ekf = upd.x, upd.P
    
    ekf_estimates.append(x_ekf.copy())

ekf_estimates = np.array(ekf_estimates)

print(f"EKF final position: ({ekf_estimates[-1, 0]:.2f}, {ekf_estimates[-1, 2]:.2f})")
print(f"True final position: ({true_states[-1, 0]:.2f}, {true_states[-1, 2]:.2f})")

In [ ]:
# Compare KF (with position measurements) vs EKF (with radar measurements)
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(true_states[:, 0], true_states[:, 2], 'g-', linewidth=2, label='True trajectory')
ax.plot(estimates[:, 0], estimates[:, 2], 'b--', linewidth=1.5, label='KF (position meas.)')
ax.plot(ekf_estimates[:, 0], ekf_estimates[:, 2], 'r:', linewidth=1.5, label='EKF (radar meas.)')

# Plot radar measurement rays (every 10th)
for i in range(0, len(radar_measurements), 10):
    r, theta = radar_measurements[i]
    ax.plot([0, r*np.cos(theta)], [0, r*np.sin(theta)], 'k-', alpha=0.2, linewidth=0.5)

ax.scatter([0], [0], c='k', s=100, marker='^', label='Radar')
ax.set_xlabel('X position (m)')
ax.set_ylabel('Y position (m)')
ax.set_title('Linear KF vs Extended KF Comparison')
ax.legend()
ax.axis('equal')
plt.show()

## 3. Unscented Kalman Filter (UKF)

The UKF uses sigma points to capture the mean and covariance of a nonlinear transformation more accurately than the EKF's first-order linearization.

Key advantages:
- No Jacobian computation required
- Better accuracy for highly nonlinear systems
- Captures up to second-order moments

In [ ]:
# Run UKF with radar measurements
x_ukf = np.array([1.0, 0.0, 1.0, 0.0])
P_ukf = np.eye(4) * 10.0

ukf_estimates = [x_ukf.copy()]

for z in radar_measurements:
    # Predict
    pred = ukf_predict(x_ukf, P_ukf, f_cv, Q)
    x_pred, P_pred = pred.x, pred.P
    
    # Update
    upd = ukf_update(x_pred, P_pred, z, h_radar, R_radar)
    x_ukf, P_ukf = upd.x, upd.P
    
    ukf_estimates.append(x_ukf.copy())

ukf_estimates = np.array(ukf_estimates)

print(f"UKF final position: ({ukf_estimates[-1, 0]:.2f}, {ukf_estimates[-1, 2]:.2f})")

In [ ]:
# Compare EKF vs UKF
ekf_error = np.sqrt((ekf_estimates[1:, 0] - true_states[1:, 0])**2 + 
                     (ekf_estimates[1:, 2] - true_states[1:, 2])**2)
ukf_error = np.sqrt((ukf_estimates[1:, 0] - true_states[1:, 0])**2 + 
                     (ukf_estimates[1:, 2] - true_states[1:, 2])**2)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(time, ekf_error, 'r-', label=f'EKF (RMSE: {np.sqrt(np.mean(ekf_error**2)):.3f})')
ax.plot(time, ukf_error, 'b-', label=f'UKF (RMSE: {np.sqrt(np.mean(ukf_error**2)):.3f})')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position Error (m)')
ax.set_title('EKF vs UKF Position Error')
ax.legend()
plt.show()

## 4. Square-Root Kalman Filter

Square-root filters maintain the Cholesky factorization of the covariance matrix, providing:
- Guaranteed positive semi-definiteness
- Improved numerical stability
- Better conditioning for ill-conditioned problems

In [ ]:
# Run Square-Root KF
x_sr = np.array([0.0, 0.0, 0.0, 0.0])
S_sr = np.linalg.cholesky(np.eye(4) * 10.0)  # Square root of covariance

sr_estimates = [x_sr.copy()]

for z in measurements:
    # Predict
    pred = sr_kf_predict(x_sr, S_sr, F, Q)
    x_pred, S_pred = pred.x, pred.S
    
    # Update
    upd = sr_kf_update(x_pred, S_pred, z, H, R)
    x_sr, S_sr = upd.x, upd.S
    
    sr_estimates.append(x_sr.copy())

sr_estimates = np.array(sr_estimates)

# Compare with standard KF
kf_rmse = np.sqrt(np.mean((estimates[1:, :2] - true_states[1:, [0, 2]])**2))
sr_rmse = np.sqrt(np.mean((sr_estimates[1:, :2] - true_states[1:, [0, 2]])**2))

print(f"Standard KF position RMSE: {kf_rmse:.4f}")
print(f"Square-Root KF position RMSE: {sr_rmse:.4f}")
print(f"Difference: {abs(kf_rmse - sr_rmse):.6f} (should be very small)")

## 5. Filter Comparison Summary

| Filter | Linearity | Jacobian Required | Numerical Stability | Computational Cost |
|--------|-----------|-------------------|---------------------|--------------------|
| KF | Linear only | N/A | Good | Low |
| EKF | Nonlinear | Yes | Moderate | Low |
| UKF | Nonlinear | No | Good | Medium |
| SR-KF | Linear only | N/A | Excellent | Low |
| SR-UKF | Nonlinear | No | Excellent | Medium |

## Exercises

1. **Modify the process noise**: Increase `q` to 1.0 and observe how the filter responds.
2. **Add measurement outliers**: Inject a few large measurement errors and implement a simple outlier rejection.
3. **Implement a turning target**: Use the coordinated turn model for a maneuvering target.
4. **Compare filter consistency**: Compute and plot the Normalized Estimation Error Squared (NEES).

## References

1. Bar-Shalom, Y., Li, X. R., & Kirubarajan, T. (2001). *Estimation with Applications to Tracking and Navigation*.
2. Simon, D. (2006). *Optimal State Estimation: Kalman, H∞, and Nonlinear Approaches*.
3. Julier, S. J., & Uhlmann, J. K. (2004). Unscented filtering and nonlinear estimation. *Proceedings of the IEEE*.